# Mixture Density Networks

本文主要参考了以下资料简单了解 在不确定性量化方面的神经网络模型Mixture Density Networks

- [A Hitchhiker’s Guide to Mixture Density Networks](https://towardsdatascience.com/a-hitchhikers-guide-to-mixture-density-networks-76b435826cca)
- [Mixture Density Networks: Probabilistic Regression for Uncertainty Estimation](https://deep-and-shallow.com/2021/03/20/mixture-density-networks-probabilistic-regression-for-uncertainty-estimation/)
- [Mixture Density Networks](https://cedar.buffalo.edu/~srihari/CSE574/Chap5/Chap5.6-MixDensityNetworks.pdf)
- [sagelywizard/pytorch-mdn](https://github.com/sagelywizard/pytorch-mdn)
- [BrandonSmithJ/MDN](https://github.com/BrandonSmithJ/MDN)
- [Verification tools for probabilistic forecasts of continuous hydrological variables](https://doi.org/10.5194/hess-11-1267-2007)
- [Uncertainty Estimation with Deep Learning for Rainfall–Runoff Modelling](https://doi.org/10.5194/hess-2021-154)

## 介绍

不确定性就在我们身边。它存在于我们做出的每一个决定、采取的每一个行动中，是我们做出的每一个决定的关键因素。在我们为未来规划的业务决策中尤其如此。理想情况下，我们的决定应该充分了解并考虑（或至少承认）潜在的不确定性。但尽管如此，我们在业务中使用的预测模型很多都忽略了不确定性。

假设您是 Google Play 商店的经理，并且 Google Pixel 5a 即将发布。HQ 会根据他们的 ML 模型向您发送预测，并表示他们预计在发布的第一周内将售出 100 台。但是您知道，根据您的经验，总部的预测并不总是正确的，并且希望通过采购 100 多个股票来对冲库存。但还有多少？你怎么知道 ML 模型的预测会有多错误？换句话说，我们对这个 100 的预测有多少**信心**？模型置信度的附加信息对于做出此决定至关重要，而 HQ 的 ML 模型不会为您提供这些信息。但是，如果除了这个 100 的预测之外，该模型还为您提供了**不确定性的度量**– 像预期概率分布的标准偏差？那么就可以根据自己的风险偏好做出明智的决定，决定库存过剩的程度。

但是怎么做呢？通常，分类问题有一个额外的优势，因为我们在顶部设置了逻辑函数，这让我们对模型的置信度有了一些了解（尽管在技术上这不是一回事）。当涉及到回归时，我们的传统模型会给我们一个点估计。

本文主要探讨一些关于回归的，预测连续值的技术问题，以及众所周知的线性模型在某些情况下是如何受限的。然后，将展示一种称为混合密度网络 (MDN) 的神经网络变体，以规避这些限制。这样可能会更好地掌握预测的潜在不确定性。

归纳一下，我们现在希望预测具有三个基本特征。第一，预测连续量；其次，预测是在不确定性下做出的。因此，理想情况下，我们希望了解我们对预测的不确定性；第三，预测以一些输入观察为条件。理想情况下，我们不希望获得问题的单一答案，而是希望获得一系列答案来评估每个单独答案的概率。简而言之：我们正在寻找给定输入的一系列答案的概率分布。下面看看如何具体做到吧。

## 不确定性的类型

有两种主要的不确定性 - 认知 Epistemic 不确定性和任意 Aleatoric 不确定性（这块专业内的可以参考下[这篇文章](https://arxiv.org/abs/1906.04595)）。

认知不确定性描述了模型不知道的东西。这归因于模型的不足。这是可以通过拥有更多数据或增加模型复杂性来减少的不确定性。

Aleatoric Uncertainty 是数据生成过程中固有的不确定性。例如，一架由高精度设备发射的纸飞机，在保持相同的释放程度、释放速度和其他各种参数的情况下，每次试验仍然不会落在同一个地方。这种固有的变化性是任意不确定性。

一个典型的监督机器学习问题可以写成如下：

![](img/latex.png)

这里的认知不确定性来自θ，任意不确定性来自$\mathbf{x}$。通常，在数据示例稀疏的特征空间部分会发现较多的认知不确定性。在这样的 n 维空间中，可能有许多参数可以解释给定的数据点，这会导致不确定性。

![](img/image-18.png)

## 核心理念

关键创新是这样的：

在普通的神经网络回归中，我们将在最后一层有一个神经元，它被训练来预测我们感兴趣的值。如果我们改为预测**概率分布的参数**会怎样？例如，高斯分布通过其均值( $\mathbf{\mu}$) 和标准差( $\mathbf{\sigma}$) 进行参数化。因此，在最后一层不是只有一个神经元，而是有两个预测高斯分布的均值和标准差。

惊人的是如果训练得当，我们就有了**均值和标准差**，这意味着我们有整个概率分布，因此可以扩展为**不确定性的估计**。

但有个问题。以简单的线性回归预测连续量为例来说明。

给定一个输入向量x，我们希望预测y。更准确地说：我们的目标是在给定x 的情况下获得y的概率：p(y|x)。如果我们假设真实目标值数据是高斯分布（正如我们通常所做的那样，当我们最小化平方误差时），则p(y|x)的形式：

![](img/1_ACNiZjSJhkPICCffkjRSOw.png)

在实际应用中，给定X及它的参数Θ，和目标值y，以及数据集𝔻中所有（X，y）对，我们最小化平方误差项$(\mu(X, \theta) - y)^2$ 。给定数据和参数，学习函数“吐出”高斯分布 μ( x , Θ )的条件均值。它扔掉了不依赖于Θ的标准差和归一化常数。通过这样做，该模型强加了多个重要假设，这在实践中可能非常有限：

1. 数据分布为高斯分布。
> “实际的机器学习问题通常具有明显的非高斯分布”。
2. 输出分布是单峰的。因此，我们无法解释x可以产生多个有效答案的情况，而多峰分布可以捕获这些答案（如上面的示例）。
3. 假设噪声分布的 标准差 σ 是常数，因此必须不依赖于x（同方差，各向同性协方差矩阵）。同样，在现实世界中情况并非总是如此。
4. 函数 μ( x , Θ ) 是线性的，即 μ( x , Θ )= x × w + b ，其中Θ ={ w , b }。线性模型被广泛认为更具可解释性。

比如考虑下面两种情况，

![](img/1_PSMns3G-V0YY2HOcGVsd7g.png)

左图：底层函数是线性的。但是，我们观察到两个违规行为：首先，（噪声）分布的std不是恒定的。其次，噪声确实取决于输入。

右图：不仅是噪声分布的标准差取决于x，输出也是非线性的。此外，输出分布是多峰的。对于某些数据区域（大约 ± 8），简单平均值不是合理的解决方案。当我们要预测遵循如此复杂模式的结果时，强加先前概述的假设可能很容易产生误导。

也就是说，当我们简单假定正态分布，并训练模型来预测分布的参数时，我们对模型施加了巨大的归纳偏差。而我们试图建模的目标变量甚至可能不遵循任何参数分布。

![](img/Bimodal.png)

以双峰为例，看起来有两个高斯分布压在了一起。如果我们做一个实验，并将这种“*两个高斯分布的混合*”扩展到“ *N 个高斯分布的混合*”，这个结果混合物可以模拟各种各样的概率分布。这正是混合密度网络的代码思想。我们有许多高斯分量（均值和标准差），它们构成了网络的最后一层。并有另一个学习参数（潜在表示），它决定如何混合这些高斯分量。

## 高斯混合Gaussian Mixture的直觉

举个例子：让我们假设，我们将要预测产品的价格，比如耳机。查看市场价格的直方图，我们推断存在低价（30 美元）、中价（60 美元）和高价（120 美元）的耳机。简单的高斯分布来对数据进行建模注定会失败，这很容易从下图一个经验高斯分布的曲线的建模看出（高斯正态分布 (𝒩) 由两个值参数化：均值 (μ) 和标准差 (σ)。并根据样本均值 (μ=47.95) 和标准值将 𝒩 拟合到数据中来建模）。可以看到，偏差 (σ=27.76) 将概率分配给我们在虚构市场中没有观察到的价格区域（即价格在 90 美元到 110 美元之间）。此外，根据曲线的左侧，负价格是“合理的”。

![](img/1_Af3tuCrG1eDYEPZ4Hm8oGA.png)

生成价格分布的基础数据是高斯分布的混合。混合物是多峰的；因此，它表现出多个“峰值”。为了适应我们虚构定价数据的分布，我们可以使用不是一个，不是两个，而是三个充分混合的高斯分量。我们“只需要”选择参数（μ，σ），通过加权（⍺）对分布进行归一化，然后对它们求和（稍后会详细介绍）。混合这些组件巧妙地反映了真实数据，如下图所示。

![](img/1_CBEfTShzie89SRPdG7YoMQ.png)

In [20]:
import numpy as np
import torch
from torch import distributions as D

# Set values for the mixture
alphas = torch.Tensor([0.6, 0.3, 0.1])
means = torch.Tensor([30, 60, 120])
sigmas = torch.Tensor([5, 3, 1])

# Use the tf.probability class
gm = D.MixtureSameFamily(
    mixture_distribution=D.Categorical(probs=alphas),
    component_distribution=D.Normal(
        loc=means,       
        scale=sigmas))

# Draw 1e5 random samples from the mixture distribution
prices = gm.sample(sample_shape=torch.Size([100000]))

# Create a normal distribution with empirical mean & std. deviation
nd_empirical = D.Normal(loc=torch.mean(prices), scale=torch.std(prices))

print(torch.mean(prices))
print(torch.std(prices))

tensor(48.0682)
tensor(27.8072)


## 混合密度网络（MDN）的背景

为了解决前述局限性，通过 DNN 对分布的混合进行参数化。MDN 最初构思于 1994 年，最近出现了一系列不同的应用。例如：iOS 11 中苹果的 Siri 使用 MDN 进行语音生成；还有结合使用 MDN 和 RNN 来生成人工笔迹等（更多内容可以参考[这里的引用部分](https://towardsdatascience.com/a-hitchhikers-guide-to-mixture-density-networks-76b435826cca)）。许多现代神经网络架构都可以扩展为 MDN（Transformer、LSTM、CovNet……）。MDN 本质上可以看作是一个扩展模块，适用于各种与业务相关的任务。

MDN 概念的核心是简单、直接且有吸引力：结合深度神经网络 (DNN) 和混合分布。DNN 为多个分布提供参数，然后通过一些权重混合这些参数。这些权重也由 DNN 提供。由此产生的（多峰）条件概率分布有助于我们对现实世界数据中发现的复杂模式进行建模。因此，我们能够更好地评估我们预测的某些值的可能性。

## 混合密度网络

为了获得混合的参数，我们可以修改一个 DNN 以输出多个参数向量。那么自然地，混合密度网络由两个组件构建而成——神经网络和混合模型。

![](img/Mixture-Density-Network-The-output-of-a-neural-network-parametrizes-a-Gaussian-mixture.png)

神经网络可以是任何有效的架构，它接受输入$\mathbf{X}$并将其转换为一组学习特征（我们可以将其视为编码器或主干）。现在，让我们来看看混合模型。

### 混合模型的形式

混合模型，就像我们之前讨论的那样，是一种概率分布模型，由更简单分布的加权和构成。

在Mixture density networks最早的[Bishop 的论文](http://publications.aston.ac.uk/id/eprint/373/) 中，使用了高斯核并解释了任何概率密度函数都可以近似到任意精度，前提是混合系数和高斯参数选择正确。也就是说，从理论上讲，如果高斯混合被充分参数化（例如，给定足够的构成），它能够模拟任意概率密度。形式上，混合的条件概率定义为：

![](img/1_KFWx6dBNlMPGf_wKxFswCw.png)

分别详细说明每个参数：

- c表示对应的混合成分的索引。每个输出最多有C 个混合分量（即：分布），这是一个用户可定义的超参数。
- ⍺ 表示混合参数。将混合参数视为将不同强度的C个不同音频信号混合在一起的滑块，从而产生更丰富的输出。混合参数以输入x为条件。
- 𝒟 是要混合的相应分布（音频信号）。可以根据任务或应用程序选择分布。
- λ 表示分布 𝒟 的参数。如果我们将 𝒟 表示为高斯分布，则 λ1 对应于条件均值 μ( x )，而 λ2 对应于条件标准差σ(x)。分布可以有多个参数（例如：Bernoulli 和 Chi2 有一个，Gaussian 和 Beta 有两个，截断的高斯分布最多有四个参数）。这些是神经网络输出的参数。

将条件概率公式化为混合分布解决了与前面提及的假设相关的多个问题。首先，分布可以是任意的，因为我们理论上能够将每个分布建模为高斯的混合。其次，使用多个分布有助于我们对多峰信号进行建模。考虑前面的耳机价格示例，这显然是多峰的。三、标准差现在取决于输入，这允许我们考虑变量标准差。即使我们只使用单个高斯分布，这种优势也适用。第四，可以通过选择非线性模型来规避函数的线性问题，该模型以输入的分布参数为条件。

### 充分条件

Bishop 还提出了实现 MDN 的一些限制和方法。

1. 混合系数 ( $\pi$ 或$\alpha$) 是概率，必须小于零且总和为 1。这可以通过将混合系数的输出通过Softmax层来轻松实现。这一步很重要，因为概率的混合必须整合为一。
2. 方差 ( $\sigma$) 应严格为正。Bishop建议我们对 sigma 神经元的原始 logits 使用指数函数。然而，指数会导致数值不稳定，所以，我们可以使用简单的 softplus 激活，类似于 oneplus 激活。或者我们使用带有偏移量的 ELU 激活的变体。最近 ELU 的重要性激增，使用它较多。
3. 中心参数 ( $\mu$) 表示位置参数，这应该是平均神经元的原始对数。

### 损失函数

该网络使用标准反向传播进行端到端训练。为此，我们最小化的损失函数是负对数似然，它相当于最大似然估计。

我们已经知道是什么P(x)，这只是计算它并最大化负对数似然的问题。

## 确保稳定性的实施和技巧

现在我们知道了模型背后的理论，让我们看看如何实现它（以及一些技巧和失败模式）。实施和训练 MDN 非常困难，因为有很多事情可能会出错。但是通过之前的研究和大量的实验，可以确定一些使训练相对稳定的技巧。我们将在 PyTorch 中实现。

### 避免数值下溢

如果还记得高斯混合的pdf公式，它涉及所有由混合系数加权的pdf的总和。所以负对数似然中会看到一个指数函数，然后乘以混合系数，然后取对数。这个指数和随后的log可能会导致非常小的数字，从而导致数字下溢。所以我们尽量使用LogsumExp 技巧并在对数域中进行计算。

使用torch.logsumexp计算批次中所有样本的负对数似然，然后计算平均值。这有助于我们避免训练中的许多数值不稳定性。

### 方差参数的激活函数

Bishop 建议对方差参数使用指数激活函数。选择有其优势。指数函数趋向于正输出，在较低的一侧，它永远不会真正达到零。但实际上，它存在一些问题。指数函数增长得非常快，并且在具有高方差的数据集的情况下，训练变得不稳定。

[Axel Brando Guillaumes](https://www.semanticscholar.org/paper/Mixture-density-networks-for-distribution-and-Guillaumes/0c01928c685098d4be96c18c6243a0e8c6f66074) 提出了一个替代方案，这就是在实现中使用的 ELU 激活的修改版本。

![](img/image-1.png)

ELU 函数保留指数行为，并在值更高时恢复为线性行为。唯一的问题是指数行为是在 x 为负时。但是如果我们将这个函数向上移动 1，我们就会得到一个近似指数行为的函数。因此 Axel Brando Guillaumes 提出将其ELU()+1 用作方差参数的激活函数。由于技术上这也可以变为零，我们还在修改后的 ELU 中添加了一个 epsilon 以确保稳定性。所以最后使用的激活是：

$ELU() + 1 + \epsilon \text{ where } \epsilon \text{ is} 1e-15$

### 多重高斯和模式崩溃

当我们使用多个高斯组件并使用反向传播训练模型时，网络倾向于忽略除一个组件之外的所有组件并训练单个组件来解释数据。例如。如果我们使用两个分量，则训练一个分量将趋向于 0，另一个趋向于 1，并且将训练与变为 1 的混合分量对应的均值和方差分量来解释数据。

可以这样做：

1. 权重正则化——将 L1 或 L2 正则化应用于计算均值、方差和混合分量的神经元的权重。
2. 偏差初始化——如果我们预先计算两个高斯的可能中心，我们可以将\亩层的偏差初始化为这些中心。这已表明在训练期间对两个高斯核/分量的分离具有很强的影响。

### 混合系数的 Softmax 替代方案

Bishop 建议使用 Softmax 层将混合系数的原始对数转换为概率。还可以使用Gumbel Softmax，它提供了更清晰的概率分布。这是可取的，因为我们希望我们的模型能够在不需要时有效地分解出一个或多个组件。Softmax 仍然会为这些分配一个小概率，而 Gumbel Softmax 使这个概率更小。

![](img/Screen_Shot_2020-06-22_at_1.41.25_PM.png)

综上，如果从单层 DNN 和 ReLU 激活开始。使用隐藏层 h1( x )，继续计算混合物的参数如下所示：

![](img/1_swMoplsvlZYv35B-v__Y8g.png)

λ1 和 λ2 本身的约束取决于我们为模型选择的分布。我们必须对 Gaussian 强制执行的唯一约束是 std为σ( x) > 0. 

最终得到以下转换：

![](img/1_ElyApgqJbh9VEgkpeDtd8g.png)

约束的选择取决于分布和数据。不同的约束可能在不同的数据集上表现更好。

由于我们现在指定了参数和条件概率，因此我们能使用某种形式的梯度下降（SGD、Adagrad、Adadelta、Adam、RMSProp 等）直接最小化平均负对数似然 (NLL) 。

## MDN 的实现

在为 MDN 建立了基本理论之后，现在展示如何在 PyTorch 中实现模型。我们基本上需要两个组件：一个用于计算参数的自定义层和要最小化的损失函数。

注意损失函数：pi是权重，mu是均值，sigma是标准差。计算损失函数时候，会计算调用gaussian_probability函数，计算target在混合高斯分布中每组高斯分布的概率；然后pi加权计算混合概率，再去对数负值作为loss，也就是相当于最大化这个概率了，所以相当于是观测的对数似然最大。这样我们就大致了解是如何训练的了。

接下来是测试，训练阶段我们最后得到的是混合分布，所以测试阶段得到的也是分布，所以就针对分布采样来得到实际预测值，然后可以针对采样的值来评价预报的能力。下面的例子仍然是以均值的MSE和分布的NLL（负对数似然）来评价。不过实际应用中，在概率预报中，还有很多其他的评价方式，尤其对于时变连续变量。参考资料中的论文Verification tools for probabilistic forecasts of continuous hydrological variables 中有提及。

In [25]:
"""A module for a mixture density network layer
For more info on MDNs, see _Mixture Desity Networks_ by Bishop, 1994.
"""
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.distributions import Categorical
import math


ONEOVERSQRT2PI = 1.0 / math.sqrt(2 * math.pi)


class MDN(nn.Module):
    """A mixture density network layer
    The input maps to the parameters of a MoG probability distribution, where
    each Gaussian has O dimensions and diagonal covariance.
    Arguments:
        in_features (int): the number of dimensions in the input
        out_features (int): the number of dimensions in the output
        num_gaussians (int): the number of Gaussians per output dimensions
    Input:
        minibatch (BxD): B is the batch size and D is the number of input
            dimensions.
    Output:
        (pi, sigma, mu) (BxG, BxGxO, BxGxO): B is the batch size, G is the
            number of Gaussians, and O is the number of dimensions for each
            Gaussian. Pi is a multinomial distribution of the Gaussians. Sigma
            is the standard deviation of each Gaussian. Mu is the mean of each
            Gaussian.
    """

    def __init__(self, in_features, out_features, num_gaussians):
        super(MDN, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.num_gaussians = num_gaussians
        self.pi = nn.Sequential(
            nn.Linear(in_features, num_gaussians),
            nn.Softmax(dim=1)
        )
        self.sigma = nn.Linear(in_features, out_features * num_gaussians)
        self.mu = nn.Linear(in_features, out_features * num_gaussians)

    def forward(self, minibatch):
        pi = self.pi(minibatch)
        sigma = torch.exp(self.sigma(minibatch))
        sigma = sigma.view(-1, self.num_gaussians, self.out_features)
        mu = self.mu(minibatch)
        mu = mu.view(-1, self.num_gaussians, self.out_features)
        return pi, sigma, mu


def gaussian_probability(sigma, mu, target):
    """Returns the probability of `target` given MoG parameters `sigma` and `mu`.
    Arguments:
        sigma (BxGxO): The standard deviation of the Gaussians. B is the batch
            size, G is the number of Gaussians, and O is the number of
            dimensions per Gaussian.
        mu (BxGxO): The means of the Gaussians. B is the batch size, G is the
            number of Gaussians, and O is the number of dimensions per Gaussian.
        target (BxI): A batch of target. B is the batch size and I is the number of
            input dimensions.
    Returns:
        probabilities (BxG): The probability of each point in the probability
            of the distribution in the corresponding sigma/mu index.
    """
    target = target.unsqueeze(1).expand_as(sigma)
    ret = ONEOVERSQRT2PI * torch.exp(-0.5 * ((target - mu) / sigma)**2) / sigma
    return torch.prod(ret, 2)


def mdn_loss(pi, sigma, mu, target):
    """Calculates the error, given the MoG parameters and the target
    The loss is the negative log likelihood of the data given the MoG
    parameters.
    """
    prob = pi * gaussian_probability(sigma, mu, target)
    nll = -torch.log(torch.sum(prob, dim=1))
    return torch.mean(nll)


def sample(pi, sigma, mu):
    """Draw samples from a MoG.
    """
    # Choose which gaussian we'll sample from
    pis = Categorical(pi).sample().view(pi.size(0), 1, 1)
    # Choose a random sample, one randn for batch X output dims
    # Do a (output dims)X(batch size) tensor here, so the broadcast works in
    # the next step, but we have to transpose back.
    gaussian_noise = torch.randn(
        (sigma.size(2), sigma.size(0)), requires_grad=False)
    variance_samples = sigma.gather(1, pis).detach().squeeze()
    mean_samples = mu.detach().gather(1, pis).squeeze()
    return (gaussian_noise * variance_samples + mean_samples).transpose(0, 1)

## 在模拟数据上的应用

是时候回到我们之前的例子了。我们训练了一个简单的 MDN，它有两层，每层 200 个神经元，在线性数据集上有一个高斯分量。MDN 显示了它的优势：由于调节标准。输入分布的偏差，MDN可以适应底层数据分布的变化。它巧妙地捕捉了线性趋势（如预期），但适应了标准。根据数据中存在的不确定性增加而产生的偏差（我喜欢这张图。它看起来像一颗流星）。

![](img/1_0HhEFiF0ImnFQn0S7uPFDg.png)

为了更好地掌握结果，我们还对几个模型的平均负对数似然进行了比较。也就是说，让我们看看空模型（样本均值和样本标准差）、线性模型（线性条件均值和样本标准差）、DNN（非线性条件均值和样本标准差）以及MDN（非线性条件均值和非线性条件标准偏差）。DNN 和 MDN 使用相同的参数和训练程序。幸运的是，我们可以使用 Tensorboard 监控 MDN 的训练进度。所需要的只是对 fit 例程的回调。因此，我们不需要费心单独存储训练损失。我们正在融合！

![](img/1_p7kGFVHczDxOWl4BFG4fzg.png)

![](img/1_nPZzA_rHVLOe7XvDL3ruqQ.png)

所有模型都能够击败空模型。其余模型在 MSE 方面的表现相同，因为 MSE 假定标准差。基础分布的偏差是恒定的。我们无法充分捕捉数据的行为！NLL，包含标准。偏差，确实反映了更细微的画面。由于底层函数是线性的，因此 DNN 和线性模型的性能相同。然而，MDN能够更好地适应数据分布，从而产生最低的 NLL 值。
要从单个数据点的 MDN 概率密度导出条件均值，可以计算：

![](img/1_LpE6RUxe4_1dREZGyqnV4Q.png)

查看公式解释了结果：平均值不包含标准。偏差σ( x )。仅查看 MDN 的平均值就会丢弃有价值的信息，而我们在实际应用中可能需要这些信息。有了这个分布，我们就可以计算出更精细的数量。例如，香农熵可以作为我们确定性的指标。或者我们可以计算 f-divergences 来评估预测的相似程度。
现在让我们转向第二个 - 非线性 - 示例。我们首先使用 min-max scaler 将y转换为 DNN/MDN 的合理范围以加快学习速度。

![](img/1_qbIx4pU-XOntAsOhd3ko-Q.png)

MDN 不仅捕获了潜在的非线性，还捕获了输出的多模态和 std 的变化。偏差。数据生成分布被充分捕获。查看x = 8的条件密度，我们看到 MDN 产生了两个不相交的峰值：

![](img/1_GKB0j6XPqRpQypztQD1zAA.png)

对这些复杂分布进行建模的能力反映在 NLL 中，其中 MDN 实现了最佳 NLL。

![](img/1_d2vfb21781fMUvsQlnT1-A.png)

## 概括

评估不确定性是数据分析一个重要方面。这里重点介绍了使用 MDN 时的理论推理、实现细节以及一些提示和技巧。我们在模拟应用中展示了 MDN 的功能。由于其简单性和模块化，我们期望有广泛的应用。